In [2]:
%load_ext autoreload
%autoreload 2
import miniGPT
import torch
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
text, data, vocab_size, encoder, decoder = miniGPT.data_utils.get_shakespeare_data()
data = data.to(device,torch.long)

Vocab : 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
savedir = '/home/tau/emenier/data/GPT/DecoderGPT/Train1/'
max_block_size = 512 # what is the maximum context length for predictions?
lr = 3e-4
dtype = torch.float32
C = 768
n_heads = 12
n_layer = 9
dropout = 0.2

gpt_model = miniGPT.gpt.DecoderGPT(vocab_size, C, n_layer, n_heads, 
                    max_block_size, dropout_freq=dropout).to(device,dtype)

trainer = miniGPT.train_utils.DecoderGPTtrainer(gpt_model,lr,
                    checkpoint_path=savedir,wd=1e-5)
dic = trainer.load('last_model.trch')
train_dataset = miniGPT.train_utils.TextDataset(
                data[:int(0.9*len(data))],max_block_size)
val_dataset = miniGPT.train_utils.TextDataset(
                data[int(0.9*len(data)):],max_block_size)
miniGPT.plot_utils.plot_losses(dic)

RuntimeError: Error(s) in loading state_dict for DecoderGPT:
	Missing key(s) in state_dict: "layers.0.self_attention.bias", "layers.0.self_attention.c_attn.weight", "layers.0.self_attention.c_proj.weight", "layers.1.self_attention.bias", "layers.1.self_attention.c_attn.weight", "layers.1.self_attention.c_proj.weight", "layers.2.self_attention.bias", "layers.2.self_attention.c_attn.weight", "layers.2.self_attention.c_proj.weight", "layers.3.self_attention.bias", "layers.3.self_attention.c_attn.weight", "layers.3.self_attention.c_proj.weight", "layers.4.self_attention.bias", "layers.4.self_attention.c_attn.weight", "layers.4.self_attention.c_proj.weight", "layers.5.self_attention.bias", "layers.5.self_attention.c_attn.weight", "layers.5.self_attention.c_proj.weight", "layers.6.self_attention.bias", "layers.6.self_attention.c_attn.weight", "layers.6.self_attention.c_proj.weight", "layers.7.self_attention.bias", "layers.7.self_attention.c_attn.weight", "layers.7.self_attention.c_proj.weigh

In [ ]:
x,y = train_dataset[np.random.choice(len(train_dataset))]
x, y = x.reshape(1,*x.shape), y.reshape(1,*y.shape)
ret = trainer.gpt_model.generate(x,256)
decoded = decoder(ret[0,:].detach().cpu().numpy())
print(decoded[max_block_size-20:max_block_size] + ' ----- ' + decoded[max_block_size:])



In [ ]:
sum([p.numel() for p in trainer.gpt_model.parameters()])

In [1]:
nn.DataParallel(gpt_model)

NameError: name 'nn' is not defined